# COMS 4995_002 Deep Learning Assignment 1
Due on Thursday, Feb 8, 11:59pm

This assignment can be done in groups of at most 2 students. Everyone must submit on Courseworks individually.

Write down the UNIs of your group (if applicable)

Member 1: Name, UNI

Member 2: Name, UNI

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import scipy.misc
import glob
import sys
# you shouldn't need to make any more imports

In [47]:
class NeuralNetwork(object):
    """
    Abstraction of neural network.
    Stores parameters, activations, cached values. 
    Provides necessary functions for training and prediction. 
    """
    def __init__(self, layer_dimensions, drop_prob=0.0, reg_lambda=0.0):
        """
        Initializes the weights and biases for each layer
        :param layer_dimensions: (list) number of nodes in each layer
        :param drop_prob: drop probability for dropout layers. Only required in part 2 of the assignment
        :param reg_lambda: regularization parameter. Only required in part 2 of the assignment
        """
        np.random.seed(1)
        
        self.parameters = {}
        self.num_layers = len(layer_dimensions)
        self.drop_prob = drop_prob
        self.reg_lambda = reg_lambda
        self.lr_decay = 0.99
        self.m_beta = 0.5
        
        # init parameters
        self.parameters['W'] = {}
        self.parameters['B'] = {}
        self.parameters['Z'] = {}
        self.parameters['A'] = {}
        self.parameters['M'] = {}
        self.parameters["Wm"] = {}
        self.parameters["Bm"] = {}
        for i in range(1,len(layer_dimensions)):
            self.parameters['W'][i] = np.random.normal(size = (layer_dimensions[i],layer_dimensions[i-1])) * np.sqrt(2.0 / (layer_dimensions[i] + layer_dimensions[i-1]))
            self.parameters['B'][i] = np.random.normal(size = (layer_dimensions[i],1)) / 10
            self.parameters['Z'][i] = np.zeros((layer_dimensions[i],1))
            self.parameters['A'][i] = np.zeros((layer_dimensions[i],1))
        

    def affineForward(self, A, W, b):
        """
        Forward pass for the affine layer.
        :param A: input matrix, shape (L, S), where L is the number of hidden units in the previous layer and S is
        the number of samples
        :returns: the affine product WA + b, along with the cache required for the backward pass
        """
        Z = np.dot(W, A) + b
        
        # return affine product
        return Z
        
        

    def activationForward(self, A, activation="relu"):
        """
        Common interface to access all activation functions.
        :param A: input to the activation function
        :param prob: activation funciton to apply to A. Just "relu" for this assignment.
        :returns: activation(A)
        """ 
        if activation == "relu":
            return self.relu(A)
        elif activation == "softmax":
            return self.softmax(A)


    def relu(self, X):
        return np.maximum(0, X)
    
    def softmax(self, X):
        return np.exp(X) / np.sum(np.exp(X), axis = 0)
    
            
    def dropout(self, A, prob):
        """
        :param A: 
        :param prob: drop prob
        :returns: tuple (A, M) 
            WHERE
            A is matrix after applying dropout
            M is dropout mask, used in the backward pass
        """
        M = np.random.rand(A.shape[0], A.shape[1])
        M = (M > prob) * 1.0 / (1-prob)
        A *= M

        return A, M

    def forwardPropagation(self, X):
        """
        Runs an input X through the neural network to compute activations
        for all layers. Returns the output computed at the last layer along
        with the cache required for backpropagation.
        :returns: (tuple) AL, cache
            WHERE 
            AL is activation of last layer
            cache is cached values for each layer that
                     are needed in further steps
        """
        self.parameters["A"][0] = X
        prev_A = X
        
        for i in range(1, self.num_layers-1):
            Z = self.affineForward(prev_A, self.parameters["W"][i], self.parameters["B"][i])
            A = self.activationForward(Z)
            self.parameters["Z"][i] = Z
            self.parameters["A"][i] = A
            prev_A = A
            
            if self.drop_prob > 0:
                prev_A, self.parameters["M"][i] = self.dropout(A, self.drop_prob)
                
        
        Z = self.affineForward(prev_A, self.parameters["W"][self.num_layers-1], self.parameters["B"][self.num_layers-1])
        AL = self.activationForward(Z, "softmax")
        self.parameters["Z"][self.num_layers-1] = Z
        self.parameters["A"][self.num_layers-1] = AL
            
        return AL
    
    def costFunction(self, AL, y):
        """
        :param AL: Activation of last layer, shape (num_classes, S)
        :param y: labels, shape (S)
        :param alpha: regularization parameter
        :returns cost, dAL: A scalar denoting cost and the gradient of cost
        """
        # compute loss
        cost = -np.sum(np.log(AL[y, range(y.shape[0])])) / y.shape[0]
        
        # add L2 regularization
        if self.reg_lambda > 0:
            for l in range(1, self.num_layers):
                cost += self.reg_lambda * 0.5 * np.sum(np.square(self.parameters["W"][l]))
       
        # gradient of cost
        dAL = self.activationBackward(AL, y, "softmax") / y.shape[0]
        
        return cost, dAL

    def affineBackward(self, dA_prev, cache):
        """
        Backward pass for the affine layer.
        :param dA_prev: gradient from the next layer.
        :param cache: cache returned in affineForward
        :returns dA: gradient on the input to this layer
                 dW: gradient on the weights
                 db: gradient on the bias
        """
        A, W = cache
        dA = np.dot(W.T, dA_prev)
        dW = np.dot(dA_prev, A.T)
        db = np.expand_dims(np.sum(dA_prev, axis=1), axis=1)
        
        return dA, dW, db

    def activationBackward(self, dA, cache, activation="relu"):
        """
        Interface to call backward on activation functions.
        In this case, it's just relu. 
        """
        if activation == "relu":
            return self.relu_derivative(dA, cache)
        elif activation == "softmax":
            return self.softmax_derivative(dA, cache)
        
    def relu_derivative(self, dx, cached_x):
        dx[cached_x <= 0] = 0
        return dx
    
    def softmax_derivative(self, x, y):
        x[y, range(len(y))] -= 1
        return x

    def dropout_backward(self, dA, cache):
#         dA = np.dot(dA, cache.T)
        return dA * cache

    def backPropagation(self, dAL, Y):
        """
        Run backpropagation to compute gradients on all paramters in the model
        :param dAL: gradient on the last layer of the network. Returned by the cost function.
        :param Y: labels
        :param cache: cached values during forwardprop
        :returns gradients: dW and db for each weight/bias
        """
        gradients = {}
        gradients["dW"] = {}
        gradients["db"] = {}
        dA_prev = dAL
        for i in range(self.num_layers-1, 0, -1):
            dZ = self.activationBackward(dA_prev, self.parameters["Z"][i])
            dA, dW, db = self.affineBackward(dZ, (self.parameters["A"][i-1], self.parameters["W"][i]))
            gradients["dW"][i] = dW
            gradients["db"][i] = db
            dA_prev = dA
            if self.drop_prob > 0 and i > 1:
                #call dropout_backward
                dA_prev = self.dropout_backward(dA_prev, self.parameters["M"][i-1])
            
            
        if self.reg_lambda > 0:
            # add gradients from L2 regularization to each dW
            for l in range(1, self.num_layers):
                gradients["dW"][l] += self.reg_lambda * self.parameters["W"][l]
        
        return gradients


    def updateParameters(self, gradients, alpha):
        """
        :param gradients: gradients for each weight/bias
        :param alpha: step size for gradient descent 
        """
        for i in range(1, self.num_layers):
#             self.parameters["W"][i] -= alpha * self.parameters["Wm"][i]
#             self.parameters["B"][i] -= alpha * self.parameters["Bm"][i]
            self.parameters["W"][i] -= alpha * gradients["dW"][i]
            self.parameters["B"][i] -= alpha * gradients["db"][i]
    
    def rotate_right(self, X):
        X_rot = []
        for i in range(len(X)):
            cur = np.reshape(X[i], (32,32,3))
            cur_rot = np.rot90(cur)
            X_rot.append(cur_rot.flatten())
        return X_rot
    
    def reflect(self, X, h, v):
        X_flip = []
        for i in range(len(X)):
            cur_flip = np.reshape(X[i], (32,32,3))
            if h:
                cur_flip = np.flipud(cur_flip)
            if v:
                cur_flip = np.fliplr(cur_flip)
            X_flip.append(cur_flip.flatten())
        return X_flip
    
    def data_aug(self, X, y, rotation=True, reflection=True):
        X_expand, y_expand = X.T, y.T
        
        if rotation:
            X_90 = self.rotate_right(X_expand)
            X_180 = self.rotate_right(X_90)
            X_270 = self.rotate_right(X_180)
            X_expand = np.vstack((X_expand, X_90, X_180, X_270))
            y_expand = y_expand + [y_expand for i in range(3)]
        
        if reflection:
            X_flip_h = self.reflect(X_expand, 1, 0)
            X_flip_v = self.reflect(X_expand, 0, 1)
            X_flip_hv = self.reflect(X_expand, 1, 1)
            X_expand = np.vstack((X_expand, X_flip_h, X_flip_v, X_flip_hv))
            y_expand = y_expand + [y_expand for i in range(3)]
            
        return X_expand.T, y_expand.T
        
        
    def train(self, X, y, iters=1000, alpha=0.0001, batch_size=100, print_every=100):
        """
        :param X: input samples, each column is a sample
        :param y: labels for input samples, y.shape[0] must equal X.shape[1]
        :param iters: number of training iterations
        :param alpha: step size for gradient descent
        :param batch_size: number of samples in a minibatch
        :param print_every: no. of iterations to print debug info after
        """
#         X_expand, y_expand = self.data_aug(X, y)
        X_expand, y_expand = X, y
        
        num_samples = X_expand.shape[1]
        dev_size = int(num_samples*0.9)
        indices = np.random.permutation(num_samples)
        dev_idx, val_idx = indices[:dev_size], indices[dev_size:]
        X_dev, y_dev = X_expand[:,dev_idx], y_expand[dev_idx]
        X_val, y_val = X_expand[:,val_idx], y_expand[val_idx]
        
        for i in range(0, iters):
            # get minibatch
            cur_X, cur_y = self.get_batch(X_dev, y_dev, batch_size)
            
            # forward prop
            AL = self.forwardPropagation(cur_X)
            
            # compute loss
            cost, dAL = self.costFunction(AL, cur_y)
            
            # compute gradients
            gradients = self.backPropagation(dAL, cur_y)

            # update weights and biases based on gradient
            if i == 0:
                for l in range(1, self.num_layers):
                    self.parameters["Wm"][l] = gradients["dW"][l]
                    self.parameters["Bm"][l] = gradients["db"][l]
            else:
                for l in range(1, self.num_layers):
                    self.parameters["Wm"][l] = (1-self.m_beta)*gradients["dW"][l]+self.m_beta*self.parameters["Wm"][l]
                    self.parameters["Bm"][l] = (1-self.m_beta)*gradients["db"][l]+self.m_beta*self.parameters["Bm"][l]
            
            self.updateParameters(gradients, alpha)
#             alpha = alpha * np.power(self.lr_decay, i+1)

            if i % print_every == 0:
                # print cost, train and validation set accuracies
                pred_dev = self.predict(X_dev)
                acc_dev = np.sum(pred_dev == y_dev) / len(y_dev)
                pred_val = self.predict(X_val)
                acc_val = np.sum(pred_val == y_val) / len(y_val)
                print("Iter: ", i, "cost: ", cost, "acc_dev: ", acc_dev, "acc_val: ", acc_val)
            
        pred_dev = self.predict(X_dev)
        acc_dev = np.sum(pred_dev == y_dev) / len(y_dev)
        pred_val = self.predict(X_val)
        acc_val = np.sum(pred_val == y_val) / len(y_val)
        print("Iter: ", iters, "cost: ", cost, "acc_dev: ", acc_dev, "acc_val: ", acc_val)
                
    def predict(self, X):
        """
        Make predictions for each sample
        """
        y_soft = self.forwardPropagation(X)
        y_pred = np.argmax(y_soft, axis=0)

        return y_pred

    def get_batch(self, X, y, batch_size):
        """
        Return minibatch of samples and labels
        
        :param X, y: samples and corresponding labels
        :parma batch_size: minibatch size
        :returns: (tuple) X_batch, y_batch
        """
        num_samples = X.shape[1]
        batch_index = np.random.randint(0, num_samples, size=batch_size)
        X_batch = X[:,batch_index]
        y_batch = y[batch_index]
            
        return X_batch, y_batch

In [2]:
# Helper functions, DO NOT modify this

def get_img_array(path):
    """
    Given path of image, returns it's numpy array
    """
    return scipy.misc.imread(path)

def get_files(folder):
    """
    Given path to folder, returns list of files in it
    """
    filenames = [file for file in glob.glob(folder+'*/*')]
    filenames.sort()
    return filenames

def get_label(filepath, label2id):
    """
    Files are assumed to be labeled as: /path/to/file/999_frog.png
    Returns label for a filepath
    """
    tokens = filepath.split('/')
    label = tokens[-1].split('_')[1][:-4]
    if label in label2id:
        return label2id[label]
    else:
        sys.exit("Invalid label: " + label)

In [3]:
# Functions to load data, DO NOT change these

def get_labels(folder, label2id):
    """
    Returns vector of labels extracted from filenames of all files in folder
    :param folder: path to data folder
    :param label2id: mapping of text labels to numeric ids. (Eg: automobile -> 0)
    """
    files = get_files(folder)
    y = []
    for f in files:
        y.append(get_label(f,label2id))
    return np.array(y)

def one_hot(y, num_classes=10):
    """
    Converts each label index in y to vector with one_hot encoding
    """
    y_one_hot = np.zeros((y.shape[0], num_classes))
    y_one_hot[y] = 1
    return y_one_hot.T

def get_label_mapping(label_file):
    """
    Returns mappings of label to index and index to label
    The input file has list of labels, each on a separate line.
    """
    with open(label_file, 'r') as f:
        id2label = f.readlines()
        id2label = [l.strip() for l in id2label]
    label2id = {}
    count = 0
    for label in id2label:
        label2id[label] = count
        count += 1
    return id2label, label2id

def get_images(folder):
    """
    returns numpy array of all samples in folder
    each column is a sample resized to 30x30 and flattened
    """
    files = get_files(folder)
    images = []
    count = 0
    
    for f in files:
        count += 1
        if count % 10000 == 0:
            print("Loaded {}/{}".format(count,len(files)))
        img_arr = get_img_array(f)
        img_arr = img_arr.flatten() / 255.0
        images.append(img_arr)
    X = np.column_stack(images)

    return X

def get_train_data(data_root_path):
    """
    Return X and y
    """
    train_data_path = data_root_path + 'train'
    id2label, label2id = get_label_mapping(data_root_path+'labels.txt')
    print(label2id)
    X = get_images(train_data_path)
    y = get_labels(train_data_path, label2id)
    return X, y

def save_predictions(filename, y):
    """
    Dumps y into .npy file
    """
    np.save(filename, y)

In [4]:
# Load the data
data_root_path = '../HW1/hw1_dataset/'
X_train, y_train = get_train_data(data_root_path) # this may take a few minutes
X_test = get_images(data_root_path + 'test')
print('Data loading done')

{'airplane': 0, 'automobile': 1, 'bird': 2, 'cat': 3, 'deer': 4, 'dog': 5, 'frog': 6, 'horse': 7, 'ship': 8, 'truck': 9}
Loaded 10000/50000
Loaded 20000/50000
Loaded 30000/50000
Loaded 40000/50000
Loaded 50000/50000
Loaded 10000/10000
Data loading done


## Part 1

#### Simple fully-connected deep neural network

In [48]:
layer_dimensions = [X_train.shape[0], 512, 512, 256, 256, 128, 64, 32, 10]  # including the input and output layers

In [ ]:
NN = NeuralNetwork(layer_dimensions)
NN.train(X_train, y_train, iters=15000, alpha=0.05, batch_size=100, print_every=1000)

Iter:  0 cost:  1.59531348064 acc_dev:  0.526155555556 acc_val:  0.5234


In [ ]:
y_predicted = NN.predict(X_test)
save_predictions('ans1-uni', y_predicted)

In [ ]:
# test if your numpy file has been saved correctly
loaded_y = np.load('ans1-uni.npy')
print(loaded_y.shape)
loaded_y[:10]

## Part 2: Improving the performance

In [5]:
class NeuralNetwork(object):
    """
    Abstraction of neural network.
    Stores parameters, activations, cached values. 
    Provides necessary functions for training and prediction. 
    """
    def __init__(self, layer_dimensions, drop_prob=0.0, reg_lambda=0.0):
        """
        Initializes the weights and biases for each layer
        :param layer_dimensions: (list) number of nodes in each layer
        :param drop_prob: drop probability for dropout layers. Only required in part 2 of the assignment
        :param reg_lambda: regularization parameter. Only required in part 2 of the assignment
        """
        np.random.seed(1)
        
        self.parameters = {}
        self.num_layers = len(layer_dimensions)
        self.drop_prob = drop_prob
        self.reg_lambda = reg_lambda
        self.lr_decay = 0.9999
        self.m_beta = 0.5
        
        # init parameters
        self.parameters['W'] = {}
        self.parameters['B'] = {}
        self.parameters['Z'] = {}
        self.parameters['A'] = {}
        self.parameters['M'] = {}
        self.parameters["Wm"] = {}
        self.parameters["Bm"] = {}
        for i in range(1,len(layer_dimensions)):
            self.parameters['W'][i] = np.random.normal(size = (layer_dimensions[i],layer_dimensions[i-1])) * np.sqrt(2.0 / (layer_dimensions[i] + layer_dimensions[i-1]))
            self.parameters['B'][i] = np.random.normal(size = (layer_dimensions[i],1)) / 10
            self.parameters['Z'][i] = np.zeros((layer_dimensions[i],1))
            self.parameters['A'][i] = np.zeros((layer_dimensions[i],1))
        

    def affineForward(self, A, W, b):
        """
        Forward pass for the affine layer.
        :param A: input matrix, shape (L, S), where L is the number of hidden units in the previous layer and S is
        the number of samples
        :returns: the affine product WA + b, along with the cache required for the backward pass
        """
        Z = np.dot(W, A) + b
        
        # return affine product
        return Z
        
        

    def activationForward(self, A, activation="relu"):
        """
        Common interface to access all activation functions.
        :param A: input to the activation function
        :param prob: activation funciton to apply to A. Just "relu" for this assignment.
        :returns: activation(A)
        """ 
        if activation == "relu":
            return self.relu(A)
        elif activation == "softmax":
            return self.softmax(A)


    def relu(self, X):
        return np.maximum(0, X)
    
    def softmax(self, X):
        return np.exp(X) / np.sum(np.exp(X), axis = 0)
    
            
    def dropout(self, A, prob):
        """
        :param A: 
        :param prob: drop prob
        :returns: tuple (A, M) 
            WHERE
            A is matrix after applying dropout
            M is dropout mask, used in the backward pass
        """
        M = np.random.rand(A.shape[0], A.shape[1])
        M = (M > prob) * 1.0 / (1-prob)
        A *= M

        return A, M

    def forwardPropagation(self, X):
        """
        Runs an input X through the neural network to compute activations
        for all layers. Returns the output computed at the last layer along
        with the cache required for backpropagation.
        :returns: (tuple) AL, cache
            WHERE 
            AL is activation of last layer
            cache is cached values for each layer that
                     are needed in further steps
        """
        self.parameters["A"][0] = X
        prev_A = X
        
        for i in range(1, self.num_layers-1):
            Z = self.affineForward(prev_A, self.parameters["W"][i], self.parameters["B"][i])
            A = self.activationForward(Z)
            self.parameters["Z"][i] = Z
            self.parameters["A"][i] = A
            prev_A = A
            
            if self.drop_prob > 0:
                prev_A, self.parameters["M"][i] = self.dropout(A, self.drop_prob)
                
        
        Z = self.affineForward(prev_A, self.parameters["W"][self.num_layers-1], self.parameters["B"][self.num_layers-1])
        AL = self.activationForward(Z, "softmax")
        self.parameters["Z"][self.num_layers-1] = Z
        self.parameters["A"][self.num_layers-1] = AL
            
        return AL
    
    def costFunction(self, AL, y):
        """
        :param AL: Activation of last layer, shape (num_classes, S)
        :param y: labels, shape (S)
        :param alpha: regularization parameter
        :returns cost, dAL: A scalar denoting cost and the gradient of cost
        """
        # compute loss
        cost = -np.sum(np.log(AL[y, range(y.shape[0])])) / y.shape[0]
        
        # add L2 regularization
        if self.reg_lambda > 0:
            for l in range(1, self.num_layers):
                cost += self.reg_lambda * 0.5 * np.sum(np.square(self.parameters["W"][l]))
       
        # gradient of cost
        dAL = self.activationBackward(AL, y, "softmax") / y.shape[0]
        
        return cost, dAL

    def affineBackward(self, dA_prev, cache):
        """
        Backward pass for the affine layer.
        :param dA_prev: gradient from the next layer.
        :param cache: cache returned in affineForward
        :returns dA: gradient on the input to this layer
                 dW: gradient on the weights
                 db: gradient on the bias
        """
        A, W = cache
        dA = np.dot(W.T, dA_prev)
        dW = np.dot(dA_prev, A.T)
        db = np.expand_dims(np.sum(dA_prev, axis=1), axis=1)
        
        return dA, dW, db

    def activationBackward(self, dA, cache, activation="relu"):
        """
        Interface to call backward on activation functions.
        In this case, it's just relu. 
        """
        if activation == "relu":
            return self.relu_derivative(dA, cache)
        elif activation == "softmax":
            return self.softmax_derivative(dA, cache)
        
    def relu_derivative(self, dx, cached_x):
        dx[cached_x <= 0] = 0
        return dx
    
    def softmax_derivative(self, x, y):
        x[y, range(len(y))] -= 1
        return x

    def dropout_backward(self, dA, cache):
#         dA = np.dot(dA, cache.T)
        return dA * cache

    def backPropagation(self, dAL, Y):
        """
        Run backpropagation to compute gradients on all paramters in the model
        :param dAL: gradient on the last layer of the network. Returned by the cost function.
        :param Y: labels
        :param cache: cached values during forwardprop
        :returns gradients: dW and db for each weight/bias
        """
        gradients = {}
        gradients["dW"] = {}
        gradients["db"] = {}
        dA_prev = dAL
        for i in range(self.num_layers-1, 0, -1):
            dZ = self.activationBackward(dA_prev, self.parameters["Z"][i])
            dA, dW, db = self.affineBackward(dZ, (self.parameters["A"][i-1], self.parameters["W"][i]))
            gradients["dW"][i] = dW
            gradients["db"][i] = db
            dA_prev = dA
            if self.drop_prob > 0 and i > 1:
                #call dropout_backward
                dA_prev = self.dropout_backward(dA_prev, self.parameters["M"][i-1])
            
            
        if self.reg_lambda > 0:
            # add gradients from L2 regularization to each dW
            for l in range(1, self.num_layers):
                gradients["dW"][l] += self.reg_lambda * self.parameters["W"][l]
        
        return gradients


    def updateParameters(self, gradients, alpha):
        """
        :param gradients: gradients for each weight/bias
        :param alpha: step size for gradient descent 
        """
        for i in range(1, self.num_layers):
            self.parameters["W"][i] -= alpha * self.parameters["Wm"][i]
            self.parameters["B"][i] -= alpha * self.parameters["Bm"][i]
#             self.parameters["W"][i] -= alpha * gradients["dW"][i]
#             self.parameters["B"][i] -= alpha * gradients["db"][i]
    
    def rotate_right(self, X):
        X_rot = []
        for i in range(len(X)):
            cur = np.reshape(X[i], (32,32,3))
            cur_rot = np.rot90(cur)
            X_rot.append(cur_rot.flatten())
        return X_rot
    
    def reflect(self, X, h, v):
        X_flip = []
        for i in range(len(X)):
            cur_flip = np.reshape(X[i], (32,32,3))
            if h:
                cur_flip = np.flipud(cur_flip)
            if v:
                cur_flip = np.fliplr(cur_flip)
            X_flip.append(cur_flip.flatten())
        return X_flip
    
    def data_aug(self, X, y, rotation=True, reflection=True):
        X_expand, y_expand = X.T, y.T
        
        if rotation:
            X_90 = self.rotate_right(X_expand)
            X_180 = self.rotate_right(X_90)
            X_270 = self.rotate_right(X_180)
            X_expand = np.vstack((X_expand, X_90, X_180, X_270))
            y_expand = y_expand + [y_expand for i in range(3)]
        
        if reflection:
            X_flip_h = self.reflect(X_expand, 1, 0)
            X_flip_v = self.reflect(X_expand, 0, 1)
            X_flip_hv = self.reflect(X_expand, 1, 1)
            X_expand = np.vstack((X_expand, X_flip_h, X_flip_v, X_flip_hv))
            y_expand = y_expand + [y_expand for i in range(3)]
            
        return X_expand.T, y_expand.T
        
        
    def train(self, X, y, iters=1000, alpha=0.0001, batch_size=100, print_every=100):
        """
        :param X: input samples, each column is a sample
        :param y: labels for input samples, y.shape[0] must equal X.shape[1]
        :param iters: number of training iterations
        :param alpha: step size for gradient descent
        :param batch_size: number of samples in a minibatch
        :param print_every: no. of iterations to print debug info after
        """
#         X_expand, y_expand = self.data_aug(X, y)
        X_expand, y_expand = X, y
        
        num_samples = X_expand.shape[1]
        dev_size = int(num_samples*0.9)
        indices = np.random.permutation(num_samples)
        dev_idx, val_idx = indices[:dev_size], indices[dev_size:]
        X_dev, y_dev = X_expand[:,dev_idx], y_expand[dev_idx]
        X_val, y_val = X_expand[:,val_idx], y_expand[val_idx]
        
        for i in range(0, iters):
            # get minibatch
            cur_X, cur_y = self.get_batch(X_dev, y_dev, batch_size)
            
            # forward prop
            AL = self.forwardPropagation(cur_X)
            
            # compute loss
            cost, dAL = self.costFunction(AL, cur_y)
            
            # compute gradients
            gradients = self.backPropagation(dAL, cur_y)

            # update weights and biases based on gradient
            if i == 0:
                for l in range(1, self.num_layers):
                    self.parameters["Wm"][l] = gradients["dW"][l]
                    self.parameters["Bm"][l] = gradients["db"][l]
            else:
                for l in range(1, self.num_layers):
                    self.parameters["Wm"][l] = (1-self.m_beta)*gradients["dW"][l]+self.m_beta*self.parameters["Wm"][l]
                    self.parameters["Bm"][l] = (1-self.m_beta)*gradients["db"][l]+self.m_beta*self.parameters["Bm"][l]
            
            self.updateParameters(gradients, alpha)
            alpha = alpha * self.lr_decay

            if i % print_every == 0:
                # print cost, train and validation set accuracies
                pred_dev = self.predict(X_dev)
                acc_dev = np.sum(pred_dev == y_dev) / len(y_dev)
                pred_val = self.predict(X_val)
                acc_val = np.sum(pred_val == y_val) / len(y_val)
                print("Iter: ", i, "cost: ", cost, "acc_dev: ", acc_dev, "acc_val: ", acc_val)
            
        pred_dev = self.predict(X_dev)
        acc_dev = np.sum(pred_dev == y_dev) / len(y_dev)
        pred_val = self.predict(X_val)
        acc_val = np.sum(pred_val == y_val) / len(y_val)
        print("Iter: ", iters, "cost: ", cost, "acc_dev: ", acc_dev, "acc_val: ", acc_val)
                
    def predict(self, X):
        """
        Make predictions for each sample
        """
        y_soft = self.forwardPropagation(X)
        y_pred = np.argmax(y_soft, axis=0)

        return y_pred

    def get_batch(self, X, y, batch_size):
        """
        Return minibatch of samples and labels
        
        :param X, y: samples and corresponding labels
        :parma batch_size: minibatch size
        :returns: (tuple) X_batch, y_batch
        """
        num_samples = X.shape[1]
        batch_index = np.random.randint(0, num_samples, size=batch_size)
        X_batch = X[:,batch_index]
        y_batch = y[batch_index]
            
        return X_batch, y_batch

In [6]:
layer_dimensions = [X_train.shape[0], 1024, 512, 256, 256, 128, 128, 64, 10]  # including the input and output layers

In [7]:
NN2 = NeuralNetwork(layer_dimensions, drop_prob=0.001, reg_lambda=0.001)
NN2.train(X_train, y_train, iters=10000, alpha=0.1, batch_size=100, print_every=1000)

Iter:  0 cost:  3.90161469109 acc_dev:  0.113511111111 acc_val:  0.1088
Iter:  1000 cost:  2.90922301035 acc_dev:  0.407644444444 acc_val:  0.3924
Iter:  2000 cost:  2.69215355146 acc_dev:  0.469733333333 acc_val:  0.4522
Iter:  3000 cost:  2.43343325455 acc_dev:  0.509266666667 acc_val:  0.4644
Iter:  4000 cost:  2.14614946843 acc_dev:  0.535044444444 acc_val:  0.486
Iter:  5000 cost:  1.99671885987 acc_dev:  0.538733333333 acc_val:  0.4806
Iter:  6000 cost:  1.7954285334 acc_dev:  0.588377777778 acc_val:  0.5142
Iter:  7000 cost:  1.94442494396 acc_dev:  0.599888888889 acc_val:  0.5118
Iter:  8000 cost:  1.71262722971 acc_dev:  0.621511111111 acc_val:  0.5244
Iter:  9000 cost:  1.35392729318 acc_dev:  0.655377777778 acc_val:  0.5336
Iter:  10000 cost:  1.43662644039 acc_dev:  0.6692 acc_val:  0.5222


In [ ]:
y_predicted2 = NN2.predict(X)
save_predictions(y_predicted, 'ans2-uni')

Write down results for Part 2 here:
...